# User Exploration
I'll need a users for my demo presentation of how my model will function in a real world scenario. 
In this notebook I'll look at some potential users and build profiles on them 

In [1]:
import pandas as pd
import numpy as np
import pickle

#### Functions

In [2]:
def filter_predictions_to_dataframe(predictions, businesses_df, user_id, open_only=True, categories=None, city=None, n=10):
    """
    Filter the predictions to only include businesses that are open, match specific categories,
    or are in a certain city, and return a DataFrame with their details and the estimated ratings.

    :param predictions: The list of predictions for all users and businesses.
    :param businesses_df: The DataFrame containing business data.
    :param user_id: The user ID for whom to filter the top N ratings.
    :param open_only: Boolean indicating if only businesses that are open should be returned.
    :param categories: List of categories to filter by.
    :param city: The city to filter by.
    :param n: The number of top recommendations to retrieve.
    :return: A DataFrame with business details and predicted ratings.
    """
    # Convert the predictions to a DataFrame
    preds_df = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions],
                            columns=['user_id', 'business_id', 'predicted_rating'])

    # Filter for the specific user
    user_preds_df = preds_df[preds_df['user_id'] == user_id]

    # Join with the business data
    merged_df = user_preds_df.merge(businesses_df, on='business_id', how='left')

    # Apply the filters
    if open_only:
        merged_df = merged_df[merged_df['is_open'] == 1]
    if categories is not None:
        merged_df = merged_df[merged_df['categories'].apply(lambda cats: any(cat in cats for cat in categories))]
    if city is not None:
        merged_df = merged_df[merged_df['city'] == city]

    # Sort by predicted rating and select the top N
    filtered_preds = merged_df.nlargest(n, 'predicted_rating')

    # Select the specified columns
    final_columns = ['business_id', 'name', 'address', 'city', 'state', 'postal_code',
                     'latitude', 'longitude', 'stars', 'predicted_rating', 'review_count',
                     'categories', 'hours']
    result_df = filtered_preds[final_columns]

    return result_df

# Example usage:
# Replace 'specific_user_id' with the actual user ID
# Specify any additional filters you want to apply

# top_businesses_df = filter_predictions_to_dataframe(
#     loaded_predictions,
#     businesses_df,
#     'specific_user_id',
#     open_only=True,
#     categories=['Japanese', 'Sushi'],
#     city='Philadelphia',
#     n=10
# )


## Restaurants

#### Importing Data

In [3]:
# Loading user ratings for restaurants
restaurants_ratings_df = pd.read_csv('../Data/ratings/restaurant_ratings.csv')
restaurants_ratings_df.shape

(129505, 3)

In [4]:
# Loading business info for restaurants_df
restaurants_df = pd.read_csv('../Data/filtered_data_sets/restaurants.csv')
restaurants_df.shape

(4323, 13)

In [5]:
# Loading user info for users that have rated the restaurants above
restaurant_users_df = pd.read_csv('../Data/filtered_data_sets/restaurant_users.csv')
restaurant_users_df.shape

(5515, 4)

In [6]:
%%time
# Loading predicitons generated from our previous model
with open('../Models/r_predictions.pkl', 'rb') as f:
    r_predictions = pickle.load(f)

CPU times: user 44.3 s, sys: 21.5 s, total: 1min 5s
Wall time: 1min 14s


In [7]:
# Convert predictions to a DataFrame
r_predictions_df = pd.DataFrame(r_predictions)
# Dropping unnecessary columns
r_predictions_df = r_predictions_df.drop(['r_ui', 'details'], axis=1)

r_predictions_df.shape

(23300761, 3)

In [8]:
restaurants_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"['Sushi Bars', 'Restaurants', 'Japanese']","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
1,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"['Korean', 'Restaurants']","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
2,tMkwHmWFUEXrC9ZduonpTg,The Green Pheasant,215 1st Ave S,Nashville,TN,37201,36.159886,-86.773197,4.0,161,0,"['Restaurants', 'Japanese', 'Seafood']","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2..."
3,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,3.0,41,0,"['Pizza', 'Restaurants', 'Salad', 'Soup']","{'Tuesday': '17:0-21:30', 'Wednesday': '17:0-1..."
4,TLZ3-eDPLhUzfsWO4ad6Ug,Mahony's Po-Boys & Seafood,901 Iberville St,New Orleans,LA,70112,29.955415,-90.070062,4.0,382,1,"['Restaurants', 'Seafood', 'Cajun/Creole']","{'Monday': '0:0-0:0', 'Thursday': '15:0-20:0',..."


In [9]:
restaurants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   4323 non-null   object 
 1   name          4323 non-null   object 
 2   address       4319 non-null   object 
 3   city          4323 non-null   object 
 4   state         4323 non-null   object 
 5   postal_code   4323 non-null   int64  
 6   latitude      4323 non-null   float64
 7   longitude     4323 non-null   float64
 8   stars         4323 non-null   float64
 9   review_count  4323 non-null   int64  
 10  is_open       4323 non-null   int64  
 11  categories    4323 non-null   object 
 12  hours         3677 non-null   object 
dtypes: float64(3), int64(3), object(7)
memory usage: 439.2+ KB


In [10]:
restaurant_users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        5515 non-null   object 
 1   name           5515 non-null   object 
 2   review_count   5515 non-null   int64  
 3   average_stars  5515 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 172.5+ KB


In [11]:
restaurant_users_df.head(20)

,user_id,name,review_count,average_stars
0,NIhcRW6DWvk1JQhDhXwgOQ,Lia,2288,3.69
1,IpLRJY4CP3fXtlEd8Y4GFQ,Robyn,518,2.95
2,RgDVC3ZUBqpEe6Y1kPhIpw,Monica,1282,4.17
3,LwZJFLGxQwjjeOgpqTJnfw,Lucas,157,3.63
4,OJorsQjotS1dIv--IVifCA,Cindy,252,3.77
5,w-zD8Ln3XZszM82AfVrspg,Robin,90,4.11
6,KrIL3TIOJI-tjvU6BlcA-g,Elisabeth,141,4.22
7,FlhZYUEhfpfm0Ynxacftwg,Kate,94,4.10
8,gRL9s6ENxV1qermRYDWjPQ,Moira,182,3.80
9,0G-QF457q_0Z_jKqh6xWiA,Joi,2009,4.10


#### User Profile: Thomas

In [12]:
# Saving ratings for user: Thomas
thomas_ratings = restaurants_ratings_df[restaurants_ratings_df["user_id"].str.contains("ySdQOT6sGt13SNsFjECEPA")]

In [13]:
thomas_ratings.head()

,user_id,business_id,stars
126916,ySdQOT6sGt13SNsFjECEPA,0oSSjekU-3GR8gselReWnA,5.0
126917,ySdQOT6sGt13SNsFjECEPA,3ldp8B52AOzNSFnPMAY0ww,5.0
126918,ySdQOT6sGt13SNsFjECEPA,4_-IcMpkF_sBRHomWZHNzA,5.0
126919,ySdQOT6sGt13SNsFjECEPA,6CpCnFzpyOV4ymhNP1AOGw,5.0
126920,ySdQOT6sGt13SNsFjECEPA,C-JAsI-tgmf_eRmMXTp9Ew,5.0


In [14]:
thomas_ratings['stars'].mean()

4.944444444444445

In [15]:
thomas_restaurants = restaurants_df[restaurants_df.business_id.isin(thomas_ratings.business_id )]
thomas_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
12,dChRGpit9fM_kZK5pafNyA,The Love,130 S 18th St,Philadelphia,PA,19103,39.950656,-75.170899,4.0,618,1,"['Restaurants', 'American (New)', 'Breakfast &...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:0', ..."
260,d_tRshM-w6S4QxE4VVi8tQ,Jones,700 Chestnut St,Philadelphia,PA,19106,39.949314,-75.152525,3.5,1141,0,"['American (New)', 'Restaurants', 'Southern', ...","{'Monday': '16:0-21:0', 'Tuesday': '16:0-21:0'..."
631,4_-IcMpkF_sBRHomWZHNzA,Bistrot La Minette,623 S 6th St,Philadelphia,PA,19147,39.941462,-75.152385,4.5,619,1,"['Restaurants', 'French', 'Breakfast & Brunch']","{'Wednesday': '17:30-21:30', 'Thursday': '17:3..."
836,0oSSjekU-3GR8gselReWnA,Butcher and Singer,1500 Walnut St,Philadelphia,PA,19102,39.949334,-75.166176,4.5,1290,1,"['Restaurants', 'Steakhouses', 'American (Trad...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
1057,e3Y3hDpwHc9RmQlJtIgHuw,Lacroix Restaurant at The Rittenhouse,210 W Rittenhouse Sq,Philadelphia,PA,19103,39.949913,-75.173645,4.5,434,1,"['Restaurants', 'Breakfast & Brunch', 'Brazili...","{'Monday': '17:30-22:30', 'Tuesday': '17:30-21..."


In [16]:
thomas_restaurants['categories'].value_counts()

['Restaurants', 'American (New)', 'Breakfast & Brunch', 'Burgers']                                       1
['Steakhouses', 'Restaurants', 'American (Traditional)']                                                 1
['Mexican', 'Honduran', 'Restaurants']                                                                   1
['Restaurants', 'Asian Fusion', 'Chinese']                                                               1
['Restaurants', 'American (New)']                                                                        1
['Restaurants', 'Tacos', 'Tex-Mex', 'Mexican']                                                           1
['American (New)', 'Restaurants']                                                                        1
['Restaurants', 'Barbeque']                                                                              1
['American (Traditional)', 'Restaurants', 'Fast Food', 'Burgers']                                        1
['American (New)', 'Restaurants', 'So

From the data we have learned the Thomas is from Philadelphia, tends to rate businesses highly with an average 4.94 rating, and shows a preference toward american cuisine, steakhouses, barbaque, burgers. 

##### Generating Top 10 Predicted Ratings for Thomas in Philidelphia

In [17]:
top_businesses_df = filter_predictions_to_dataframe(
    r_predictions,
    restaurants_df,
    'ySdQOT6sGt13SNsFjECEPA',
    city='Philadelphia',
    n=10
)

# Display the DataFrame
top_businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
48,p-MSh95si_sDdih94gyKuw,Forsythia,233 Chestnut St,Philadelphia,PA,19106,39.948707,-75.145394,4.5,5.0,123,"['Restaurants', 'French', 'Modern European', '...","{'Wednesday': '17:0-22:0', 'Thursday': '17:0-2..."
125,V4Dr3ragKHKeUab96miyMA,Mole Poblano Restaurant,1144 S 9th St,Philadelphia,PA,19147,39.935220,-75.158815,4.5,5.0,149,"['Mexican', 'Restaurants']","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
166,wUnLSg_GKfEIQ5CQQ770_g,Vetri Cucina,1312 Spruce St,Philadelphia,PA,19107,39.946644,-75.163187,4.5,5.0,489,"['Italian', 'Cuban', 'Restaurants']","{'Monday': '18:0-21:30', 'Tuesday': '18:0-21:3..."
196,pDurCski6kcl8mcXmpcKKw,Ace of Steaks,3020 S Broad St,Philadelphia,PA,19145,39.912641,-75.172892,4.5,5.0,31,"['Restaurants', 'Cheesesteaks', 'Sandwiches', ...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
273,HNI6qvswn1KYybgJVqUWGg,Marinucci's Deli - Mayfair,2852 Saint Vincent St,Philadelphia,PA,19149,40.041504,-75.054122,4.5,5.0,130,"['Delis', 'Restaurants']","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
356,TKPAyOWcexkpVHPCdYTNmQ,Spuntino Wood Fired Pizza,701 N 2nd St,Philadelphia,PA,19123,39.962006,-75.140950,4.5,5.0,209,"['Pizza', 'Restaurants']","{'Tuesday': '12:0-21:0', 'Wednesday': '12:0-21..."
361,6085NRg7QH3vXpc50F4UHQ,Chiquita's Pizzeria & Mexican Grill,746 Snyder Ave,Philadelphia,PA,19148,39.922904,-75.159634,5.0,5.0,48,"['Mexican', 'Pizza', 'Restaurants', 'Italian']","{'Tuesday': '11:0-23:0', 'Wednesday': '11:0-23..."
364,U7HYUH8SqZO6OQMNKCr5kQ,Porco's Porchetteria,2204 Washington Ave,Philadelphia,PA,19146,39.939305,-75.180117,4.5,5.0,84,"['Restaurants', 'Sandwiches']","{'Wednesday': '10:0-18:0', 'Thursday': '10:0-1..."
365,fEqiXG_B-fn__w0aeF3nBQ,Laurel,1617 E Passyunk Ave,Philadelphia,PA,19148,39.929415,-75.163715,4.5,5.0,326,"['French', 'American (New)', 'Restaurants']","{'Wednesday': '17:30-22:0', 'Thursday': '17:30..."
368,mlrzM-jUkKot28XJhScuuA,Goldie,1526 Sansom St,Philadelphia,PA,19102,39.950227,-75.166864,4.5,5.0,514,"['Falafel', 'Middle Eastern', 'Mediterranean',...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-18:0', ..."


In [18]:
top_businesses_df = filter_predictions_to_dataframe(
    r_predictions,
    restaurants_df,
    'ySdQOT6sGt13SNsFjECEPA',
    categories=['Barbeque'],
    city='Philadelphia',
    n=5
)

# Display the DataFrame
top_businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
842,fB_VJEJTIMyWv5hC9T-xwA,So Korean Grill,"6201 N Front St, Ste 120",Philadelphia,PA,19120,40.045317,-75.117653,4.5,5.000000,77,"['Korean', 'Barbeque', 'Restaurants']","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
3992,zRnFxKLKdn0ZJ2_DapEQWA,The Lucky Well - Spring Arts,990 Spring Garden St,Philadelphia,PA,19123,39.961265,-75.154128,5.0,5.000000,27,"['Chicken Shop', 'Barbeque', 'Restaurants']","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2..."
2009,5Q82x5hIwfWSF3XEeINkgQ,Dienner's Bar-B-Q Chicken,12TH And Arch St,Philadelphia,PA,19102,39.953773,-75.159079,4.5,4.994207,58,"['Food Stands', 'Restaurants', 'Barbeque']","{'Tuesday': '8:0-15:0', 'Wednesday': '8:0-15:0..."
3414,YS18M5ox3x0tK6uulfydrw,ZIG ZAG BBQ,2111 E York St,Philadelphia,PA,19125,39.982291,-75.128143,4.5,4.970367,24,"['Restaurants', 'Barbeque']","{'Wednesday': '15:30-20:0', 'Thursday': '15:30..."
149,l8gAoQqqVfphPe9jmIJZ3g,K-POT Korean BBQ & Hot Pot,789 Franklin Mills Cir,Philadelphia,PA,19154,40.092469,-74.962939,4.5,4.958059,349,"['Korean', 'Barbeque', 'Hot Pot', 'Restaurants']","{'Monday': '12:0-22:30', 'Tuesday': '12:0-22:3..."


I noticed that with users who have a narrow spread in ratings meaning they only tend to rate the business they like and give 4 or 5 star. With out data on what users dislike the predictions tend to skew higher and it becomes difficult to differentiate between all these predicted 5 star restaurants. The user would have to use a category filter like 'Barbeque' to narrow his reccomondations. 

#### User Profile: Monica

In [19]:
# Saving ratings for user: Monica
monica_ratings = restaurants_ratings_df[restaurants_ratings_df["user_id"].str.contains('RgDVC3ZUBqpEe6Y1kPhIpw')]
monica_ratings.shape

(72, 3)

In [20]:
monica_ratings.head()

,user_id,business_id,stars
57913,RgDVC3ZUBqpEe6Y1kPhIpw,-DGsnMlRrR_tv8avrpQLQw,4.0
57914,RgDVC3ZUBqpEe6Y1kPhIpw,-DGsnMlRrR_tv8avrpQLQw,3.0
57915,RgDVC3ZUBqpEe6Y1kPhIpw,-DGsnMlRrR_tv8avrpQLQw,4.0
57916,RgDVC3ZUBqpEe6Y1kPhIpw,0oSSjekU-3GR8gselReWnA,5.0
57917,RgDVC3ZUBqpEe6Y1kPhIpw,0oSSjekU-3GR8gselReWnA,5.0


In [21]:
monica_ratings['stars'].mean()

4.013888888888889

In [22]:
monica_restaurants = restaurants_df[restaurants_df.business_id.isin(monica_ratings.business_id )]
monica_restaurants.head(60)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
161,pj8Y-5vfbRitnnm2xJFH9w,Jamaican Jerk Hut,1436 South St,Philadelphia,PA,19146,39.944275,-75.166787,2.5,276,1,"['Caribbean', 'Restaurants']","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
260,d_tRshM-w6S4QxE4VVi8tQ,Jones,700 Chestnut St,Philadelphia,PA,19106,39.949314,-75.152525,3.5,1141,0,"['American (New)', 'Restaurants', 'Southern', ...","{'Monday': '16:0-21:0', 'Tuesday': '16:0-21:0'..."
312,eaDZlSuVS0EY67Ke6pRP6Q,Penang,117 N 10th St,Philadelphia,PA,19107,39.954044,-75.155980,3.5,1112,1,"['Malaysian', 'Asian Fusion', 'Cambodian', 'Ch...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
327,JUlsvVAvZvGHWFfkKm0nlg,El Camino Real,1040 N 2nd St,Philadelphia,PA,19123,39.967300,-75.140398,3.5,1014,1,"['Mexican', 'Tex-Mex', 'Restaurants', 'Barbeque']","{'Monday': '0:0-0:0', 'Tuesday': '11:0-23:0', ..."
425,cXSyVvOr9YRN9diDkaWs0Q,Honey's Sit-N-Eat,800 N 4th St,Philadelphia,PA,19123,39.963990,-75.144482,4.0,1396,1,"['Southern', 'Restaurants', 'American (Traditi...","{'Monday': '9:0-14:0', 'Tuesday': '9:0-14:0', ..."
501,zRjUMHQJ5gAmFhcXZtLacA,Vietnam Restaurant,221 N 11th St,Philadelphia,PA,19107,39.956330,-75.157069,4.0,421,1,"['Vietnamese', 'Restaurants']","{'Monday': '11:30-20:0', 'Tuesday': '11:30-20:..."
532,jziliEq8Zum-EynD5v-Hvg,Chifa,707 Chestnut St,Philadelphia,PA,19106,39.949786,-75.152623,4.0,444,0,"['Peruvian', 'Chinese', 'Restaurants', 'Asian ...",NaN
603,4CBk3DBUgl3V9zbVBKsIqQ,Moshi Moshi,108 S 18th St,Philadelphia,PA,19103,39.951392,-75.170620,3.0,40,0,"['Japanese', 'Sushi Bars', 'Restaurants']",NaN
614,ErJ9NtMXt-WJZV_TVZ08Ag,Green Garden,237 S 10th St,Philadelphia,PA,19107,39.947166,-75.157393,3.5,77,1,"['Sushi Bars', 'Chinese', 'Soup', 'Seafood', '...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
746,imd0v6mixJCBnqGr6RV3hQ,Rami's,40th and Locust,Philadelphia,PA,19195,39.952956,-75.202838,4.5,16,0,"['Middle Eastern', 'Food Stands', 'Restaurants']",NaN


In [23]:
monica_restaurants['categories'].value_counts()

['Restaurants', 'Mexican']                                                                               3
['Caribbean', 'Restaurants']                                                                             1
['Breakfast & Brunch', 'Restaurants']                                                                    1
['Asian Fusion', 'Thai', 'Soup', 'Restaurants', 'French', 'Salad']                                       1
['Modern European', 'American (New)', 'Restaurants']                                                     1
['Restaurants', 'Steakhouses', 'Seafood']                                                                1
['Pizza', 'Italian', 'Restaurants']                                                                      1
['Restaurants', 'Diners', 'Breakfast & Brunch', 'Sandwiches']                                            1
['Restaurants', 'Chinese', 'Vietnamese']                                                                 1
['Restaurants', 'Food Stands', 'Break

From the data we have learned the Monica is from Philadelphia, is a prolific reviewer  and is honest in her ratings with an average 4.013 star rating, and enjoys a variety of flavors. Maybe eats out for lunch and dinner on a regular basis

##### Generating Top 10 Predicted Ratings for Monica in Philidelphia

In [24]:
top_businesses_df = filter_predictions_to_dataframe(
    r_predictions,
    restaurants_df,
    'RgDVC3ZUBqpEe6Y1kPhIpw',
    city='Philadelphia',
    n=10
)

# Display the DataFrame
top_businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
1265,fq1yCVBgBB7s6V-D68NO1g,Cafe Mi Quang,3324 Kensington Ave,Philadelphia,PA,19134,39.998429,-75.110254,5.0,5.000000,69,"['Restaurants', 'Vietnamese']","{'Monday': '8:30-19:0', 'Wednesday': '8:30-19:..."
952,8j5bQ0nDwoGiFaTQDNCQTg,Jack's Place,7167 Hegerman St,Philadelphia,PA,19135,40.028190,-75.035811,4.5,4.907303,78,"['Sandwiches', 'Restaurants', 'Salad', 'Cheese...","{'Tuesday': '10:0-18:0', 'Wednesday': '10:0-18..."
476,yCUQ6csyVOR9krwQH9RIbQ,Haveli Virasat,4051 Market St,Philadelphia,PA,19104,39.957624,-75.203795,4.5,4.906054,68,"['Gluten-Free', 'Halal', 'Restaurants', 'India...","{'Tuesday': '11:30-21:30', 'Wednesday': '11:30..."
730,oZzN706lKoL4faaTK739xA,Restaurant Ambra,705 S 4th St,Philadelphia,PA,19147,39.940324,-75.149300,5.0,4.865813,61,"['Italian', 'Restaurants']","{'Thursday': '18:0-21:0', 'Friday': '18:0-21:0..."
2226,-NG3pA9LEXCxt0Fjd_aYYA,Linda's Vegetarian Village Restaurant,6381 Germantown Ave,Philadelphia,PA,19144,40.045853,-75.182057,4.5,4.861465,45,"['Vegetarian', 'Live/Raw Food', 'Vegan', 'Rest...","{'Wednesday': '10:0-18:0', 'Thursday': '10:0-1..."
2442,2VgdPHszJdhOxzVFBbUPAA,Down North Pizza,2804 W Lehigh Ave,Philadelphia,PA,19132,39.996740,-75.177223,4.5,4.858461,55,"['Chicken Wings', 'Pizza', 'Restaurants', 'Pop...","{'Thursday': '12:0-20:30', 'Friday': '12:0-21:..."
2395,AxcyIxsjJbRNG2RFTguavw,Mike's Hot Dog Cart,24th & Passyunk Ave,Philadelphia,PA,19146,39.922381,-75.186929,5.0,4.842695,22,"['Food Stands', 'Restaurants']",NaN
2559,X3NS8516o015Z1zcOwrqmQ,Jansen,7402 Germantown Ave,Philadelphia,PA,19119,40.062707,-75.193182,4.5,4.813192,139,"['American (New)', 'Restaurants']","{'Wednesday': '17:0-20:0', 'Thursday': '17:0-2..."
358,fEqiXG_B-fn__w0aeF3nBQ,Laurel,1617 E Passyunk Ave,Philadelphia,PA,19148,39.929415,-75.163715,4.5,4.779275,326,"['French', 'American (New)', 'Restaurants']","{'Wednesday': '17:30-22:0', 'Thursday': '17:30..."
1753,kT8IlV47kz1rz2lTuNyO1w,Christie's Deli,"1822 Spring Garden St, Unit B",Philadelphia,PA,19130,39.963081,-75.169360,5.0,4.779204,152,"['Restaurants', 'Breakfast & Brunch', 'Delis',...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."


Her recommondations include a lot of highly rated restaurants that she has yet to visit. In contrast with Thomas her prediction scores are alot more spread out not as many perfect 5 star restaurants which make sorting recommondations easier. 

##### Exporting Monica Predictions
This file will be what I will load onto stream lit rather than loading the entire set of predictions

In [38]:
monica_r_predictions_df = r_predictions_df[r_predictions_df['uid'].str.contains('RgDVC3ZUBqpEe6Y1kPhIpw')]
monica_r_predictions_df.shape

(4189, 3)

In [39]:
monica_r_predictions_df.to_csv('../Data/user_predictions/monica_r_predictions', index=False)

#### User Profile: Lucas

In [19]:
# Saving ratings for user: Lucas
lucas_ratings = restaurants_ratings_df[restaurants_ratings_df["user_id"].str.contains('LwZJFLGxQwjjeOgpqTJnfw')]
lucas_ratings.shape

(17, 3)

In [20]:
lucas_ratings.head()

,user_id,business_id,stars
47110,LwZJFLGxQwjjeOgpqTJnfw,1b5mnK8bMnnju_cvU65GqQ,4.0
47111,LwZJFLGxQwjjeOgpqTJnfw,2Z8YwDy6fUUTFFSSNXbLBg,4.0
47112,LwZJFLGxQwjjeOgpqTJnfw,3CFVBCfjdCvESS1ogBv21A,5.0
47113,LwZJFLGxQwjjeOgpqTJnfw,8CCLze2jLi_0hWHkv0IkDg,5.0
47114,LwZJFLGxQwjjeOgpqTJnfw,8SqvtaRu9likNDAfAU-HBQ,1.0


In [21]:
lucas_ratings['stars'].mean()

3.6470588235294117

In [23]:
lucas_restaurants = restaurants_df[restaurants_df.business_id.isin(lucas_ratings.business_id )]
lucas_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
33,8SqvtaRu9likNDAfAU-HBQ,Panda Express,455 Opry Mills Dr,Nashville,TN,37214,36.202728,-86.694489,3.0,15,1,"['Fast Food', 'Restaurants', 'Chinese']","{'Monday': '11:0-19:0', 'Tuesday': '11:0-19:0'..."
1187,o37egH3jB6egfgCbHIdoCA,The Italian Kitchen,217 4th Ave N,Nashville,TN,37219,36.164193,-86.779476,2.5,92,1,"['Restaurants', 'Sandwiches', 'Italian', 'Pizz...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
1380,3CFVBCfjdCvESS1ogBv21A,DeSano Pizza Bakery,115 16th Ave S,Nashville,TN,37203,36.152762,-86.792568,4.5,992,1,"['Pizza', 'Restaurants']","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1534,8CCLze2jLi_0hWHkv0IkDg,Joe Natural's,209 10th Ave S,Nashville,TN,37203,36.154729,-86.782348,3.5,10,0,"['American (New)', 'Restaurants']",NaN
1807,f9yBZy6IRQwX1yyy_9X6xA,IHOP,6800 Charlotte Pike,Nashville,TN,37209,36.134910,-86.889030,2.0,78,1,"['Restaurants', 'American (New)', 'American (T...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-22:0', '..."


In [24]:
lucas_restaurants['categories'].value_counts()

['Restaurants', 'Pizza']                                                                                                 2
['Fast Food', 'Restaurants', 'Chinese']                                                                                  1
['Restaurants', 'Sandwiches', 'Italian', 'Pizza', 'American (New)']                                                      1
['Pizza', 'Restaurants']                                                                                                 1
['American (New)', 'Restaurants']                                                                                        1
['Restaurants', 'American (New)', 'American (Traditional)', 'Breakfast & Brunch', 'Burgers']                             1
['Vegan', 'Restaurants', 'Live/Raw Food']                                                                                1
['Mexican', 'Restaurants', 'Latin American', 'Seafood']                                                                  1
['Restaurants', 

From the data we have learned the Lucas lives in Nashville, maybe goes out for lunch frequently, he gives his honest rating with an average rating of 3.64, and shows a preference toward foriegn cuisines. 

In [27]:
top_businesses_df = filter_predictions_to_dataframe(
    r_predictions,
    restaurants_df,
    'LwZJFLGxQwjjeOgpqTJnfw',
    city='Philadelphia',
    n=10
)

# Display the DataFrame
top_businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
1294,fq1yCVBgBB7s6V-D68NO1g,Cafe Mi Quang,3324 Kensington Ave,Philadelphia,PA,19134,39.998429,-75.110254,5.0,4.576434,69,"['Restaurants', 'Vietnamese']","{'Monday': '8:30-19:0', 'Wednesday': '8:30-19:..."
974,8j5bQ0nDwoGiFaTQDNCQTg,Jack's Place,7167 Hegerman St,Philadelphia,PA,19135,40.028190,-75.035811,4.5,4.494720,78,"['Sandwiches', 'Restaurants', 'Salad', 'Cheese...","{'Tuesday': '10:0-18:0', 'Wednesday': '10:0-18..."
491,yCUQ6csyVOR9krwQH9RIbQ,Haveli Virasat,4051 Market St,Philadelphia,PA,19104,39.957624,-75.203795,4.5,4.459097,68,"['Gluten-Free', 'Halal', 'Restaurants', 'India...","{'Tuesday': '11:30-21:30', 'Wednesday': '11:30..."
2481,2VgdPHszJdhOxzVFBbUPAA,Down North Pizza,2804 W Lehigh Ave,Philadelphia,PA,19132,39.996740,-75.177223,4.5,4.443410,55,"['Chicken Wings', 'Pizza', 'Restaurants', 'Pop...","{'Thursday': '12:0-20:30', 'Friday': '12:0-21:..."
746,oZzN706lKoL4faaTK739xA,Restaurant Ambra,705 S 4th St,Philadelphia,PA,19147,39.940324,-75.149300,5.0,4.417139,61,"['Italian', 'Restaurants']","{'Thursday': '18:0-21:0', 'Friday': '18:0-21:0..."
2599,X3NS8516o015Z1zcOwrqmQ,Jansen,7402 Germantown Ave,Philadelphia,PA,19119,40.062707,-75.193182,4.5,4.394842,139,"['American (New)', 'Restaurants']","{'Wednesday': '17:0-20:0', 'Thursday': '17:0-2..."
2265,-NG3pA9LEXCxt0Fjd_aYYA,Linda's Vegetarian Village Restaurant,6381 Germantown Ave,Philadelphia,PA,19144,40.045853,-75.182057,4.5,4.388967,45,"['Vegetarian', 'Live/Raw Food', 'Vegan', 'Rest...","{'Wednesday': '10:0-18:0', 'Thursday': '10:0-1..."
2433,AxcyIxsjJbRNG2RFTguavw,Mike's Hot Dog Cart,24th & Passyunk Ave,Philadelphia,PA,19146,39.922381,-75.186929,5.0,4.350462,22,"['Food Stands', 'Restaurants']",NaN
3992,zRnFxKLKdn0ZJ2_DapEQWA,The Lucky Well - Spring Arts,990 Spring Garden St,Philadelphia,PA,19123,39.961265,-75.154128,5.0,4.349908,27,"['Chicken Shop', 'Barbeque', 'Restaurants']","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2..."
1791,kT8IlV47kz1rz2lTuNyO1w,Christie's Deli,"1822 Spring Garden St, Unit B",Philadelphia,PA,19130,39.963081,-75.169360,5.0,4.346560,152,"['Restaurants', 'Breakfast & Brunch', 'Delis',...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."


##### Exporting Lucas Predictions
This file will be what I will load onto stream lit rather than loading the entire set of predictions

In [28]:
lucas_r_predictions_df = r_predictions_df[r_predictions_df['uid'].str.contains('LwZJFLGxQwjjeOgpqTJnfw')]
lucas_r_predictions_df.shape

(4230, 3)

In [29]:
lucas_r_predictions_df.to_csv('../Data/user_predictions/lucas_r_predictions', index=False)

## Activities

#### Importing Data

In [3]:
# Loading user ratings for activities
activity_ratings_df = pd.read_csv('../Data/ratings/activity_ratings.csv')
activity_ratings_df.shape

(2377, 3)

In [4]:
# Loading business info for restaurants_df
activities_df = pd.read_csv('../Data/filtered_data_sets/activities.csv')
activities_df.shape

(526, 13)

In [5]:
# Loading user info for users that have rated the activities above
activities_users_df = pd.read_csv('../Data/filtered_data_sets/activity_users.csv')
activities_users_df.shape

(260, 4)

In [7]:
%%time
# Loading predicitons generated from our previous model
with open('../Models/a_predictions.pkl', 'rb') as f:
    a_predictions = pickle.load(f)

CPU times: user 378 ms, sys: 6.96 ms, total: 385 ms
Wall time: 386 ms


In [8]:
# Convert predictions to a DataFrame
a_predictions_df = pd.DataFrame(a_predictions)
# Dropping unnecessary columns
a_predictions_df = a_predictions_df.drop(['r_ui', 'details'], axis=1)

a_predictions_df.shape

(88218, 3)

In [10]:
activities_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
0,uczmbBk5O3tYhGue13dCDg,New Orleans Spirit Tours,723 St Peter St,New Orleans,LA,70130.0,29.958431,-90.065173,4.0,38,1,"Hotels & Travel, Tours, Local Flavor","{'Monday': '0:0-0:0', 'Tuesday': '10:0-20:0', ..."
1,usv5G5cO314pPHRTGq2Eyw,Best Tours,NaN,Philadelphia,PA,19019.0,40.119713,-75.009710,1.0,38,1,"Tours, Walking Tours, Hotels & Travel, Food Tours",NaN
2,Qa1YQzo00sm9O0JUlwdCQw,Taste of Nawlins,504 Madison St,New Orleans,LA,70116.0,29.958015,-90.061895,4.0,178,1,"Historical Tours, Tours, Hotels & Travel, Food...","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
3,FbGjCCc7bbJVWq3Kt5lamQ,Ernie's Tours,"828 Bourbon St, Apt E",New Orleans,LA,70116.0,29.959913,-90.064281,5.0,52,1,"Walking Tours, Tours, Hotels & Travel, Histori...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-21:0', '..."
4,rh3C_JK4NAoUJ-HNStvHtw,"Historic Philadelphia, Inc","600 Walnut St, Ste 130",Philadelphia,PA,19106.0,39.948321,-75.151952,4.5,5,1,"Hotels & Travel, Tours",NaN


In [11]:
activities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   526 non-null    object 
 1   name          526 non-null    object 
 2   address       468 non-null    object 
 3   city          526 non-null    object 
 4   state         526 non-null    object 
 5   postal_code   524 non-null    float64
 6   latitude      526 non-null    float64
 7   longitude     526 non-null    float64
 8   stars         526 non-null    float64
 9   review_count  526 non-null    int64  
 10  is_open       526 non-null    int64  
 11  categories    526 non-null    object 
 12  hours         339 non-null    object 
dtypes: float64(4), int64(2), object(7)
memory usage: 53.6+ KB


In [12]:
activities_users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        260 non-null    object 
 1   name           260 non-null    object 
 2   review_count   260 non-null    int64  
 3   average_stars  260 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 8.3+ KB


In [13]:
activities_users_df.head(20)

,user_id,name,review_count,average_stars
0,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,3.74
1,LwZJFLGxQwjjeOgpqTJnfw,Lucas,157,3.63
2,0G-QF457q_0Z_jKqh6xWiA,Joi,2009,4.10
3,JITevzyEgskIJc6sgOyEgw,Kristin Marie,645,3.89
4,m07sy7eLtOjVdZ8oN9JKag,Ed,5800,3.71
5,2XKlUjHGk9z-2ckFMcBEHw,Sheena,731,3.95
6,FlXBpK_YZxLo27jcMdII1w,Mallory,1158,4.02
7,lT0jFHCIloX3bnF5eu0ERg,Mikey,262,3.96
8,cMoQqT_j_FD1gO6EEZzOIg,Mitch,1103,4.22
9,cLpT9ZCzgr7H6BWqDRs2Og,Justin,336,3.92


#### User Profile: Daniel

In [18]:
# Saving ratings for user: Daniel
daniel_ratings = activity_ratings_df[activity_ratings_df["user_id"].str.contains("j14WgRoU_-2ZE1aw1dXrJg")]
daniel_ratings.shape

(6, 3)

In [19]:
daniel_ratings.head()

,user_id,business_id,stars
1894,j14WgRoU_-2ZE1aw1dXrJg,3woevNT4gDDZWTFf94qlCg,4.0
1895,j14WgRoU_-2ZE1aw1dXrJg,6BvmLng8AmH56-eSekl3zg,4.0
1896,j14WgRoU_-2ZE1aw1dXrJg,S7tZmvSqrpH18cDE_fwVgA,4.0
1897,j14WgRoU_-2ZE1aw1dXrJg,XmYOhiKZZUhOfpaP41Go5Q,4.0
1898,j14WgRoU_-2ZE1aw1dXrJg,g_qmBN6Ta_qQLLotb02EJQ,4.0


In [20]:
daniel_ratings['stars'].mean()

3.8333333333333335

In [21]:
daniel_activities = activities_df[activities_df.business_id.isin(daniel_ratings.business_id )]
daniel_activities.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
110,3woevNT4gDDZWTFf94qlCg,Elfreth's Alley,126 Elfreths Aly,Philadelphia,PA,19106.0,39.952751,-75.142494,4.5,61,1,"Landmarks & Historical Buildings, Tours, Publi...","{'Tuesday': '10:0-17:0', 'Wednesday': '10:0-17..."
240,pCmaISSyKum22MOtPOxL-w,Philadelphia Phillies,Citizens Bank Park,Philadelphia,PA,19176.0,39.905790,-75.166301,4.5,94,1,"['Arts & Entertainment', 'Professional Sports ...",NaN
320,g_qmBN6Ta_qQLLotb02EJQ,Ben Franklin Parkway,323 N 21st St,Philadelphia,PA,19130.0,39.960435,-75.173306,4.0,16,1,"['Arts & Entertainment', 'Stadiums & Arenas', ...",NaN
344,XmYOhiKZZUhOfpaP41Go5Q,Asian Arts Initiative,1219 Vine St,Philadelphia,PA,19107.0,39.958019,-75.159268,4.5,13,1,"['Arts & Entertainment', 'Cultural Center']",NaN
376,6BvmLng8AmH56-eSekl3zg,Morris Arboretum,100 E Northwestern Ave,Philadelphia,PA,19118.0,40.089742,-75.222909,4.5,92,1,"['Arts & Entertainment', 'Botanical Gardens', ...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-16:0', ..."


In [22]:
daniel_activities['categories'].value_counts()

Landmarks & Historical Buildings, Tours, Public Services & Government, Hotels & Travel    1
['Arts & Entertainment', 'Professional Sports Teams']                                     1
['Arts & Entertainment', 'Stadiums & Arenas', 'Cultural Center']                          1
['Arts & Entertainment', 'Cultural Center']                                               1
['Arts & Entertainment', 'Botanical Gardens', 'Museums']                                  1
['Parks', 'Active Life']                                                                  1
Name: categories, dtype: int64

From the data we have learned the Daniel is from Philadelphia, rates activies with an average 3.83 rating, and enjoys a variety of activities like, sporting events, museums, and parks

##### Generating Top 10 Predicted Ratings for Daniel if he were to visit Nashville

In [24]:
top_businesses_df = filter_predictions_to_dataframe(
    a_predictions,
    activities_df,
    'j14WgRoU_-2ZE1aw1dXrJg',
    city='Nashville',
    n=10
)

# Display the DataFrame
top_businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
187,z87SlFdP6vZCjfjPkI-edQ,National Museum of African American Music,510 Broadway,Nashville,TN,37219.0,36.160531,-86.778768,5.0,4.478390,54,"['Arts & Entertainment', 'Museums']",NaN
29,2-qIaZy7A6WWXSPfyX60eA,Percy Warner Park,2500 Old Hickory Blvd,Nashville,TN,37221.0,36.055337,-86.890643,4.5,4.450640,112,"['Hiking', 'Active Life', 'Parks']",NaN
77,wsA5vD-lZHHlJrxLKvEVOw,12 South Winter Warmer,3000 Granny White Pike,Nashville,TN,37204.0,36.119781,-86.790840,5.0,4.438574,14,"['Festivals', 'Arts & Entertainment']",NaN
214,JbzvJJolDBT1614qo2Yiaw,Nelson's Green Brier Distillery,1414 Clinton St,Nashville,TN,37203.0,36.163519,-86.798211,5.0,4.401506,545,"Distilleries, Tours, Hotels & Travel, Historic...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-18:0', ..."
245,aqRLCv47YXG1qdgkHXTFdQ,Nashville Predators,501 Broadway,Nashville,TN,37203.0,36.159143,-86.778517,4.5,4.373843,54,"['Arts & Entertainment', 'Professional Sports ...",NaN
30,4Cal2AE6cr7m6nL20Rz2kQ,Ryman Auditorium,116 Rep John Lewis Way N,Nashville,TN,37219.0,36.161247,-86.778471,4.5,4.365654,574,"Local Flavor, Hotels & Travel, Music Venues, N...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
242,7s-bi0Z-OrIDJmfSfrrDOg,Beaman Park Nature Center,5911 Old Hickory Blvd,Nashville,TN,37015.0,36.262533,-86.894313,4.5,4.357199,22,"['Hiking', 'Active Life', 'Parks']",NaN
93,-9NmUeTphyS9Lq1o9MACGw,Grand Ole Opry,2804 Opryland Dr,Nashville,TN,37214.0,36.206904,-86.692294,4.5,4.339156,687,"Performing Arts, Cinema, Tours, Local Flavor, ...","{'Monday': '10:0-16:0', 'Tuesday': '17:30-21:3..."
116,feI9OMa4fBjQKHz8IxQmkg,Cavalia Odysseo,2721 McGavock Pike,Nashville,TN,37214.0,36.216446,-86.689416,5.0,4.335789,20,"['Performing Arts', 'Arts & Entertainment']",NaN
202,8qpnCNlvJN3N18dRcnD5xQ,NashTrash Tours,900 Rosa L Parks Blvd,Nashville,TN,37219.0,36.169442,-86.787940,4.5,4.320638,167,"Hotels & Travel, Tours, Local Flavor",NaN


Daniel's recommondations really seemed to strike a good balance between what is popular in Nashville and what preference Daniel has based on his Philidelphia ratings. We can see a lot of suggestions for parks, a museum, and sporting event 

#### User Profile: Kristen

In [36]:
# Saving ratings for user: Kristen
kristen_ratings = activity_ratings_df[activity_ratings_df["user_id"].str.contains('JITevzyEgskIJc6sgOyEgw')]
kristen_ratings.shape

(9, 3)

In [37]:
kristen_ratings.head()

,user_id,business_id,stars
854,JITevzyEgskIJc6sgOyEgw,1Grz73Y_zKggV14AwcGujQ,5.0
855,JITevzyEgskIJc6sgOyEgw,4Cal2AE6cr7m6nL20Rz2kQ,5.0
856,JITevzyEgskIJc6sgOyEgw,6ga2gDFtxIF93ORBHWm7Zg,5.0
857,JITevzyEgskIJc6sgOyEgw,73esU0u3hYIPPqCScAA2HQ,4.0
858,JITevzyEgskIJc6sgOyEgw,LkGbYvB4f-8n3lb5TvOY6A,2.0


In [39]:
kristen_ratings['stars'].mean()

4.333333333333333

In [40]:
kristen_activities = activities_df[activities_df.business_id.isin(kristen_ratings.business_id )]
kristen_activities.head(60)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
109,4Cal2AE6cr7m6nL20Rz2kQ,Ryman Auditorium,116 Rep John Lewis Way N,Nashville,TN,37219.0,36.161247,-86.778471,4.5,574,1,"Local Flavor, Hotels & Travel, Music Venues, N...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
140,1Grz73Y_zKggV14AwcGujQ,Fort Negley Park and Visitors Center,1100 Fort Negley Blvd,Nashville,TN,37203.0,36.142993,-86.775334,4.0,30,1,"Landmarks & Historical Buildings, Active Life,...","{'Tuesday': '12:0-16:0', 'Wednesday': '12:0-16..."
206,LkGbYvB4f-8n3lb5TvOY6A,Jackalope Brewing Company - The Den,701 8th Ave S,Nashville,TN,37203.0,36.150377,-86.779458,4.0,254,0,"Breweries, Food, Hotels & Travel, Beer Bar, Ba...",NaN
244,YA_1utFf5sYeW-T1AEkflA,Tennessee Flavors,120 White Bridge Rd,Nashville,TN,37209.0,36.134730,-86.857200,4.0,8,1,"['Festivals', 'Arts & Entertainment']",NaN
300,MuE4wrLB78kb47ASYllajQ,East Nashville Beer Festival,700 Woodland St,Nashville,TN,37206.0,36.172881,-86.760600,5.0,11,1,"['Arts & Entertainment', 'Festivals']",NaN
446,73esU0u3hYIPPqCScAA2HQ,Centennial Park,2500 W End Ave,Nashville,TN,37203.0,36.149064,-86.811993,4.5,210,1,"['Active Life', 'Parks']",NaN
486,yu9e-_S7U97kleJQw0eOzQ,Richland Creek Greenway,4601 Murphy Rd,Nashville,TN,37246.0,36.140482,-86.843183,4.5,30,1,"['Hiking', 'Active Life', 'Parks']",NaN
496,VU4jaQMZDJxKUihpnVM-LQ,Shelby Dog Park,2001 Boscobel St,Nashville,TN,37206.0,36.170845,-86.735797,4.0,33,1,"['Active Life', 'Parks', 'Dog Parks']",NaN
498,6ga2gDFtxIF93ORBHWm7Zg,Edwin Warner Park,7311 Hwy 100,Nashville,TN,37221.0,36.060805,-86.906018,4.5,45,1,"['Parks', 'Active Life', 'Hiking']","{'Tuesday': '9:0-16:0', 'Wednesday': '9:0-16:0..."


In [41]:
kristen_activities['categories'].value_counts()

Local Flavor, Hotels & Travel, Music Venues, Nightlife, Tours, Historical Tours, Arts & Entertainment, Cinema, Restaurants, Museums, American (Traditional)    1
Landmarks & Historical Buildings, Active Life, Hotels & Travel, Tours, Public Services & Government, Local Flavor, Parks                                       1
Breweries, Food, Hotels & Travel, Beer Bar, Bars, Tours, Nightlife, Beer Tours                                                                                 1
['Festivals', 'Arts & Entertainment']                                                                                                                          1
['Arts & Entertainment', 'Festivals']                                                                                                                          1
['Active Life', 'Parks']                                                                                                                                       1
['Hiking', 'Active Life', 'Parks']

From the data we have learned the Kristen is from Nashville, she is honest in her ratings with an average 4.3 star rating, and enjoys activities like parks and festivals

##### Generating Top 10 Predicted Ratings for Monica in Philidelphia

In [43]:
top_businesses_df = filter_predictions_to_dataframe(
    a_predictions,
    activities_df,
    'JITevzyEgskIJc6sgOyEgw',
    city='Nashville',
    n=10
)

# Display the DataFrame
top_businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
241,aqRLCv47YXG1qdgkHXTFdQ,Nashville Predators,501 Broadway,Nashville,TN,37203.0,36.159143,-86.778517,4.5,4.680569,54,"['Arts & Entertainment', 'Professional Sports ...",NaN
211,JbzvJJolDBT1614qo2Yiaw,Nelson's Green Brier Distillery,1414 Clinton St,Nashville,TN,37203.0,36.163519,-86.798211,5.0,4.665482,545,"Distilleries, Tours, Hotels & Travel, Historic...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-18:0', ..."
29,2-qIaZy7A6WWXSPfyX60eA,Percy Warner Park,2500 Old Hickory Blvd,Nashville,TN,37221.0,36.055337,-86.890643,4.5,4.659547,112,"['Hiking', 'Active Life', 'Parks']",NaN
239,7s-bi0Z-OrIDJmfSfrrDOg,Beaman Park Nature Center,5911 Old Hickory Blvd,Nashville,TN,37015.0,36.262533,-86.894313,4.5,4.642942,22,"['Hiking', 'Active Life', 'Parks']",NaN
184,z87SlFdP6vZCjfjPkI-edQ,National Museum of African American Music,510 Broadway,Nashville,TN,37219.0,36.160531,-86.778768,5.0,4.641020,54,"['Arts & Entertainment', 'Museums']",NaN
75,wsA5vD-lZHHlJrxLKvEVOw,12 South Winter Warmer,3000 Granny White Pike,Nashville,TN,37204.0,36.119781,-86.790840,5.0,4.636949,14,"['Festivals', 'Arts & Entertainment']",NaN
113,feI9OMa4fBjQKHz8IxQmkg,Cavalia Odysseo,2721 McGavock Pike,Nashville,TN,37214.0,36.216446,-86.689416,5.0,4.588221,20,"['Performing Arts', 'Arts & Entertainment']",NaN
258,sTj4jW7ZThn_ou3-1rKYaw,Nashville Ballet,3630 Redmon St,Nashville,TN,37209.0,36.144945,-86.827483,5.0,4.513249,7,"['Arts & Entertainment', 'Performing Arts', 'O...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
90,-9NmUeTphyS9Lq1o9MACGw,Grand Ole Opry,2804 Opryland Dr,Nashville,TN,37214.0,36.206904,-86.692294,4.5,4.512545,687,"Performing Arts, Cinema, Tours, Local Flavor, ...","{'Monday': '10:0-16:0', 'Tuesday': '17:30-21:3..."
199,8qpnCNlvJN3N18dRcnD5xQ,NashTrash Tours,900 Rosa L Parks Blvd,Nashville,TN,37219.0,36.169442,-86.787940,4.5,4.507130,167,"Hotels & Travel, Tours, Local Flavor",NaN


Her recommondations include a lot of highly rated restaurants that she has yet to visit. In contrast with Thomas her prediction scores are alot more spread out not as many perfect 5 star restaurants which make sorting recommondations easier. 

##### Exporting Monica Predictions
This file will be what I will load onto stream lit rather than loading the entire set of predictions

In [38]:
monica_r_predictions_df = r_predictions_df[r_predictions_df['uid'].str.contains('RgDVC3ZUBqpEe6Y1kPhIpw')]
monica_r_predictions_df.shape

(4189, 3)

In [39]:
monica_r_predictions_df.to_csv('../Data/user_predictions/monica_r_predictions', index=False)

#### User Profile: Lucas

In [19]:
# Saving ratings for user: Lucas
lucas_ratings = restaurants_ratings_df[restaurants_ratings_df["user_id"].str.contains('LwZJFLGxQwjjeOgpqTJnfw')]
lucas_ratings.shape

(17, 3)

In [20]:
lucas_ratings.head()

,user_id,business_id,stars
47110,LwZJFLGxQwjjeOgpqTJnfw,1b5mnK8bMnnju_cvU65GqQ,4.0
47111,LwZJFLGxQwjjeOgpqTJnfw,2Z8YwDy6fUUTFFSSNXbLBg,4.0
47112,LwZJFLGxQwjjeOgpqTJnfw,3CFVBCfjdCvESS1ogBv21A,5.0
47113,LwZJFLGxQwjjeOgpqTJnfw,8CCLze2jLi_0hWHkv0IkDg,5.0
47114,LwZJFLGxQwjjeOgpqTJnfw,8SqvtaRu9likNDAfAU-HBQ,1.0


In [21]:
lucas_ratings['stars'].mean()

3.6470588235294117

In [23]:
lucas_restaurants = restaurants_df[restaurants_df.business_id.isin(lucas_ratings.business_id )]
lucas_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,hours
33,8SqvtaRu9likNDAfAU-HBQ,Panda Express,455 Opry Mills Dr,Nashville,TN,37214,36.202728,-86.694489,3.0,15,1,"['Fast Food', 'Restaurants', 'Chinese']","{'Monday': '11:0-19:0', 'Tuesday': '11:0-19:0'..."
1187,o37egH3jB6egfgCbHIdoCA,The Italian Kitchen,217 4th Ave N,Nashville,TN,37219,36.164193,-86.779476,2.5,92,1,"['Restaurants', 'Sandwiches', 'Italian', 'Pizz...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
1380,3CFVBCfjdCvESS1ogBv21A,DeSano Pizza Bakery,115 16th Ave S,Nashville,TN,37203,36.152762,-86.792568,4.5,992,1,"['Pizza', 'Restaurants']","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
1534,8CCLze2jLi_0hWHkv0IkDg,Joe Natural's,209 10th Ave S,Nashville,TN,37203,36.154729,-86.782348,3.5,10,0,"['American (New)', 'Restaurants']",NaN
1807,f9yBZy6IRQwX1yyy_9X6xA,IHOP,6800 Charlotte Pike,Nashville,TN,37209,36.134910,-86.889030,2.0,78,1,"['Restaurants', 'American (New)', 'American (T...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-22:0', '..."


In [24]:
lucas_restaurants['categories'].value_counts()

['Restaurants', 'Pizza']                                                                                                 2
['Fast Food', 'Restaurants', 'Chinese']                                                                                  1
['Restaurants', 'Sandwiches', 'Italian', 'Pizza', 'American (New)']                                                      1
['Pizza', 'Restaurants']                                                                                                 1
['American (New)', 'Restaurants']                                                                                        1
['Restaurants', 'American (New)', 'American (Traditional)', 'Breakfast & Brunch', 'Burgers']                             1
['Vegan', 'Restaurants', 'Live/Raw Food']                                                                                1
['Mexican', 'Restaurants', 'Latin American', 'Seafood']                                                                  1
['Restaurants', 

From the data we have learned the Lucas lives in Nashville, maybe goes out for lunch frequently, he gives his honest rating with an average rating of 3.64, and shows a preference toward foriegn cuisines. 

In [27]:
top_businesses_df = filter_predictions_to_dataframe(
    r_predictions,
    restaurants_df,
    'LwZJFLGxQwjjeOgpqTJnfw',
    city='Philadelphia',
    n=10
)

# Display the DataFrame
top_businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
1294,fq1yCVBgBB7s6V-D68NO1g,Cafe Mi Quang,3324 Kensington Ave,Philadelphia,PA,19134,39.998429,-75.110254,5.0,4.576434,69,"['Restaurants', 'Vietnamese']","{'Monday': '8:30-19:0', 'Wednesday': '8:30-19:..."
974,8j5bQ0nDwoGiFaTQDNCQTg,Jack's Place,7167 Hegerman St,Philadelphia,PA,19135,40.028190,-75.035811,4.5,4.494720,78,"['Sandwiches', 'Restaurants', 'Salad', 'Cheese...","{'Tuesday': '10:0-18:0', 'Wednesday': '10:0-18..."
491,yCUQ6csyVOR9krwQH9RIbQ,Haveli Virasat,4051 Market St,Philadelphia,PA,19104,39.957624,-75.203795,4.5,4.459097,68,"['Gluten-Free', 'Halal', 'Restaurants', 'India...","{'Tuesday': '11:30-21:30', 'Wednesday': '11:30..."
2481,2VgdPHszJdhOxzVFBbUPAA,Down North Pizza,2804 W Lehigh Ave,Philadelphia,PA,19132,39.996740,-75.177223,4.5,4.443410,55,"['Chicken Wings', 'Pizza', 'Restaurants', 'Pop...","{'Thursday': '12:0-20:30', 'Friday': '12:0-21:..."
746,oZzN706lKoL4faaTK739xA,Restaurant Ambra,705 S 4th St,Philadelphia,PA,19147,39.940324,-75.149300,5.0,4.417139,61,"['Italian', 'Restaurants']","{'Thursday': '18:0-21:0', 'Friday': '18:0-21:0..."
2599,X3NS8516o015Z1zcOwrqmQ,Jansen,7402 Germantown Ave,Philadelphia,PA,19119,40.062707,-75.193182,4.5,4.394842,139,"['American (New)', 'Restaurants']","{'Wednesday': '17:0-20:0', 'Thursday': '17:0-2..."
2265,-NG3pA9LEXCxt0Fjd_aYYA,Linda's Vegetarian Village Restaurant,6381 Germantown Ave,Philadelphia,PA,19144,40.045853,-75.182057,4.5,4.388967,45,"['Vegetarian', 'Live/Raw Food', 'Vegan', 'Rest...","{'Wednesday': '10:0-18:0', 'Thursday': '10:0-1..."
2433,AxcyIxsjJbRNG2RFTguavw,Mike's Hot Dog Cart,24th & Passyunk Ave,Philadelphia,PA,19146,39.922381,-75.186929,5.0,4.350462,22,"['Food Stands', 'Restaurants']",NaN
3992,zRnFxKLKdn0ZJ2_DapEQWA,The Lucky Well - Spring Arts,990 Spring Garden St,Philadelphia,PA,19123,39.961265,-75.154128,5.0,4.349908,27,"['Chicken Shop', 'Barbeque', 'Restaurants']","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2..."
1791,kT8IlV47kz1rz2lTuNyO1w,Christie's Deli,"1822 Spring Garden St, Unit B",Philadelphia,PA,19130,39.963081,-75.169360,5.0,4.346560,152,"['Restaurants', 'Breakfast & Brunch', 'Delis',...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."


##### Exporting Lucas Predictions
This file will be what I will load onto stream lit rather than loading the entire set of predictions

In [28]:
lucas_r_predictions_df = r_predictions_df[r_predictions_df['uid'].str.contains('LwZJFLGxQwjjeOgpqTJnfw')]
lucas_r_predictions_df.shape

(4230, 3)

In [29]:
lucas_r_predictions_df.to_csv('../Data/user_predictions/lucas_r_predictions', index=False)

## Streamlit Function
The function will extract recommondations from a specific user predictions file. 

In [42]:
def filter_predictions_from_df(predictions_df, businesses_df, open_only=True, categories=None, city=None, n=10):
    """
    Filter a DataFrame of pre-filtered predictions for a specific user, join with business details, 
    and apply filters for open businesses, categories, or city, returning a DataFrame with the results.

    :param predictions_df: A DataFrame with predictions for a specific user.
    :param businesses_df: A DataFrame containing business data.
    :param open_only: Boolean indicating if only businesses that are open should be returned.
    :param categories: List of categories to filter by.
    :param city: The city to filter by.
    :param n: The number of top recommendations to retrieve.
    :return: A DataFrame with business details and predicted ratings.
    """
    # Rename 'iid' and 'est' columns to match businesses_df for merging
    predictions_df = predictions_df.rename(columns={'iid': 'business_id', 'est': 'predicted_rating'})

    # Join with the business data
    merged_df = predictions_df.merge(businesses_df, on='business_id', how='left')

    # Apply the filters
    if open_only:
        merged_df = merged_df[merged_df['is_open'] == 1]
    if categories is not None:
        merged_df = merged_df[merged_df['categories'].apply(lambda cats: any(cat in cats for cat in categories))]
    if city is not None:
        merged_df = merged_df[merged_df['city'] == city]

    # Sort by predicted rating and select the top N
    filtered_preds = merged_df.nlargest(n, 'predicted_rating')

    # Select the specified columns
    final_columns = ['business_id', 'name', 'address', 'city', 'state', 'postal_code',
                     'latitude', 'longitude', 'stars', 'predicted_rating', 'review_count',
                     'categories', 'hours']
    result_df = filtered_preds[final_columns]

    return result_df

# Example usage:
# Assuming 'monica_r_predictions_df' is the DataFrame containing predictions for the user Monica
# And 'businesses_df' is the DataFrame containing business data
# Specify any additional filters you want to apply
top_businesses_df = filter_predictions_from_df(
    monica_r_predictions_df,
    restaurants_df,
    open_only=True,
    categories=['Japanese', 'Sushi'],
    city='Philadelphia',
    n=10
)

# Display the DataFrame
top_businesses_df


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,predicted_rating,review_count,categories,hours
3516,FBof1_DJG-IMRdk6WmY2IA,Poke Burri,255 S 10th St,Philadelphia,PA,19107,39.946745,-75.157559,4.5,4.644674,53,"['Restaurants', 'Japanese']","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."
2179,j9tIldJfidEFNEMQM9vANg,Ninja Bao,"1500 Jfk Blvd, Ste 6, Two Penn Ctr Concourse L...",Philadelphia,PA,19102,39.953501,-75.165888,4.5,4.634484,49,"['Ramen', 'Izakaya', 'Asian Fusion', 'Restaura...","{'Monday': '11:0-16:30', 'Tuesday': '11:0-16:3..."
763,paNYNJ-VFDgwLY201IrXCQ,Sakana Omakasè Sushi,"616 S 2nd St, Fl 1",Philadelphia,PA,19147,39.940694,-75.145950,4.5,4.559288,157,"['Sushi Bars', 'Japanese', 'Restaurants']","{'Monday': '0:0-0:0', 'Tuesday': '18:0-23:0', ..."
2343,Ktg3ahIxk0JIkJwXAqu2ew,Hikari Sushi,"1040 N American St, Ste 701",Philadelphia,PA,19123,39.967402,-75.141153,5.0,4.527137,155,"['Restaurants', 'Sushi Bars', 'Japanese']","{'Monday': '16:30-22:0', 'Tuesday': '16:30-22:..."
750,RGzIHIhOIZzsjLRVDvSjRQ,Kinme,1117 Locust St,Philadelphia,PA,19107,39.947768,-75.160172,4.5,4.507574,110,"['Tapas/Small Plates', 'Japanese', 'Sushi Bars...","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
3140,H2c5e7PfcCg9f9Ew19-xRw,Sushi Poke,1310 South St,Philadelphia,PA,19147,39.943271,-75.163744,4.5,4.495122,29,"['Japanese', 'Restaurants', 'Sushi Bars']","{'Monday': '0:0-0:0', 'Tuesday': '11:30-22:0',..."
1082,K_s-9Wd6vXSfnxYFzhE0Kw,Kei Sushi Restaurant,1711 South St,Philadelphia,PA,19146,39.944413,-75.170739,4.5,4.480599,157,"['Sushi Bars', 'Restaurants']","{'Monday': '0:0-0:0', 'Tuesday': '11:30-21:0',..."
132,6_T2xzR74JqGCTPefAD8Tw,Morimoto,723 Chestnut St,Philadelphia,PA,19106,39.949665,-75.153273,4.5,4.472311,1914,"['Japanese', 'American (Traditional)', 'Americ...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
1178,BRl-KarobZBNuCvhz5RWKg,Ginza,1100 S Front St,Philadelphia,PA,19147,39.932450,-75.146190,4.5,4.458042,137,"['Sushi Bars', 'Ramen', 'Restaurants', 'Japane...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:30',..."
1410,8LonS_bxNmCDk3FWiG9lzA,Tomo Sushi & Ramen,228 Arch St,Philadelphia,PA,19106,39.951923,-75.144335,4.5,4.437882,193,"['Japanese', 'Restaurants', 'Ramen', 'Sushi Ba...","{'Monday': '0:0-0:0', 'Tuesday': '15:0-21:30',..."
